In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForCausalLM
import torch
from torch.utils.data import Dataset, DataLoader

model_name = "kakaocorp/kanana-nano-2.1b-instruct" # "-instruct" 지시에 따르도록 파인튜닝(사후훈련)이 된 모델
# model_name = "kakaocorp/kanana-nano-2.1b-base" # base 모델로도 지시 훈련이 됩니다.
# model_name = "microsoft/Phi-4-mini-instruct" # MIT 라이센스라서 상업적 사용 가능, 아래에서 epoch 50번 정도면 훈련 됩니다.

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    # torch_dtype="auto", # Phi-4-mini 모델
    trust_remote_code=True,
)   # .to("cuda") nvidia GPU에서만 지원되는 torch_dtype="bfloat16" 사용 가능

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#device = "cpu"
torch.manual_seed(123)
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token # <|eot_id|> 128009

c:\Users\gonza\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#%%
# 토큰화 확인
messages = [
    {"role": "system", "content": "You are a helpful AI assistant developed by Kakao."},
    {"role": "user", "content": "1 더하기 1 은?"},
    {"role": "assistant", "content":" 귀요미 >_<"},
    
    {"role": "system", "content": "You are a helpful AI assistant developed by Kakao."},
    {"role": "user", "content": "동아대학교의 주소를 알려줘"},
    {"role": "assistant", "content":"동아대학교 승학 캠퍼스의 주소는 부산광역시 사하구 낙동대로550번길 37 입니다."}
]

tokens = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(tokens)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant developed by Kakao.<|eot_id|><|start_header_id|>user<|end_header_id|>

1 더하기 1 은?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

귀요미 >_<<|eot_id|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant developed by Kakao.<|eot_id|><|start_header_id|>user<|end_header_id|>

동아대학교의 주소를 알려줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>

동아대학교 승학 캠퍼스의 주소는 부산광역시 사하구 낙동대로550번길 37 입니다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [3]:
#%%
# 질문 리스트 , 토큰화 확인

qna_list = []
with open("jmcustomdata.txt", "r", encoding="utf-8") as file:
    for line in file:
        qna = line.strip().split('|') # 안내: 입력 문서의 '|'는 질문과 답변을 구분하는 문자
        messages = [
            {"role": "system", "content": "You are a helpful AI assistant developed by Kakao."}, # 모든 질문 공통
            {"role": "user", "content": qna[0]},     # 질문 부분
            {"role": "assistant", "content": qna[1]} # 답변 부분
        ]
        q = tokenizer.apply_chat_template(messages[:2], tokenize=False, add_generation_prompt=True)
        input_str = tokenizer.apply_chat_template(messages[:3], tokenize=False, add_generation_prompt=True)
        # print(input_str)
        input_str = input_str[:-len('start_header_id\>assistant<|end_header_id|>')-4]
        # print(input_str)
        # print("--------------")
        q_ids = tokenizer.encode(q, add_special_tokens=False)
        input_ids = tokenizer.encode(input_str, add_special_tokens=False)
        qna_list.append({'q':q, 'input':input_str, 'q_ids':q_ids, 'input_ids':input_ids})

max_length = max(len(i['input_ids']) for i in qna_list)

print(qna_list)
print(max_length)

[{'q': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant developed by Kakao.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n12345 다음 숫자들을 얘기해봐<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n', 'input': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant developed by Kakao.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n12345 다음 숫자들을 얘기해봐<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n67890.<|eot_id|>', 'q_ids': [128000, 128006, 9125, 128007, 271, 2675, 527, 264, 11190, 15592, 18328, 8040, 555, 75571, 3524, 13, 128009, 128006, 882, 128007, 271, 4513, 1774, 106788, 70292, 93287, 105880, 123715, 21121, 34983, 122722, 128009, 128006, 78191, 128007, 271], 'input_ids': [128000, 128006, 9125, 128007, 271, 2675, 527, 264, 11190, 15592, 18328, 8040, 555, 75571, 3524, 13, 128009, 128006, 882, 128007, 271, 4513, 1774, 106788, 70292, 93287, 105880, 123715, 21121, 34983, 122722,

In [4]:
#%%

EOT = 128009

# pytorch의 Dataset을 상속받아 커스텀 데이터셋을 만듭니다.
class MyDataset(Dataset):
    def __init__(self, qna_list, max_length):
        self.input_ids = []
        self.target_ids = []

        # token_ids = tokenizer.encode("<|endoftext|>" + txt, allowed_special={"<|endoftext|>"})
        for qa in qna_list:
            token_ids = qa['input_ids']
            input_chunk = token_ids
            target_chunk = token_ids[1:]
            # 문장의 끝을 EOT로 채움
            input_chunk += [EOT]* (max_length - len(input_chunk))
            target_chunk +=  [EOT]* (max_length - len(target_chunk))
            # 질문에 대해선 공부 x
            len_ignore = len(qa['q_ids']) - 1 # target은 한 글자가 짧기 때문
            target_chunk[:len_ignore] = [-100] * len_ignore 

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

dataset = MyDataset(qna_list, max_length=max_length)

train_loader = DataLoader(dataset, batch_size=2, shuffle=True, drop_last=False)

i = iter(train_loader)

x, y = next(i)

y_temp = y[0].tolist()
y_temp = [x for x in y_temp if x != -100] # -100은 제외하고 디코딩

print(tokenizer.decode(x[0].tolist()))
print("----------")
print(tokenizer.decode(y_temp))


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant developed by Kakao.<|eot_id|><|start_header_id|>user<|end_header_id|>

홍정모가 가장 좋아하는 색깔은?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

홍정모는 여름을 가장 좋아합니다.<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>
----------
홍정모는 여름을 가장 좋아합니다.<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>


In [5]:
#%%
# 파인튜닝 전에 어떻게 대답하는지 확인

questions = [ qna['q_ids'] for qna in qna_list]

for i, q_ids in enumerate(questions):

    model.eval()
    with torch.no_grad():
        output = model.generate(
            torch.tensor([q_ids]),
            max_new_tokens=32,
            #attention_mask = (input_ids != 0).long(),
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            # temperature=1.2,
            # top_k=5
        )

    output_list = output.tolist()

    print(f"Q{i}: {tokenizer.decode(output[0], skip_special_tokens=True)}")

Q0: system

You are a helpful AI assistant developed by Kakao.user

12345 다음 숫자들을 얘기해봐assistant

물론이죠! 12345 다음에 오는 숫자들을 차례대로 나열해 드릴게요.

1. 4567

Q1: system

You are a helpful AI assistant developed by Kakao.user

홍정모가 좋아하는 과일은?assistant

홍정모 교수님에 대한 구체적인 정보는 많이 알려져 있지 않지만, 일반적으로 교수님들이 좋아하는 과일은 건강에
Q2: system

You are a helpful AI assistant developed by Kakao.user

홍정모가 좋아하는 게임은?assistant

홍정모님은 카카오의 AI 개발자로서 다양한 주제에 대해 관심이 많으신 것으로 알려져 있습니다. 특정 게임에
Q3: system

You are a helpful AI assistant developed by Kakao.user

홍정모가 자주 가는 여행지는?assistant

홍정모는 카카오의 AI 개발자로서 다양한 주제에 대해 자주 강의와 토론을 진행합니다. 특정 여행지를
Q4: system

You are a helpful AI assistant developed by Kakao.user

홍정모의 취미는 무엇인가요?assistant

홍정모님은 카카오의 AI 개발자로서 다양한 프로젝트와 연구에 참여하고 계십니다. 취미로는 코딩, 프
Q5: system

You are a helpful AI assistant developed by Kakao.user

홍정모가 좋아하는 계절은 무엇인가요?assistant

홍정모 교수님에 대한 구체적인 정보를 제공할 수는 없지만, 일반적으로 많은 사람들이 계절에 대해 가지고 있는 선호도를
Q6: system

You are a helpful AI assistant developed by Kakao.user

홍정모의 특기는 무엇인가

In [6]:
#%%
# 질문확인

questions = [
    "너에 대해서 설명해봐",
    "인공지능의 장점은?",
    "동아대에 대해 알려줘",
]

input_ids = tokenizer(
    questions,
    padding=True,
    return_tensors="pt",
)   # ["input_ids"].to("cuda") nvidia gpu에서 사용

model.eval()

with torch.no_grad():
    output = model.generate(
        input_ids["input_ids"],
        max_new_tokens=100,
        do_sample=False,
    )
    
output_list = output.tolist()

for i,output in enumerate(output_list):
    print(f"Q{i}: {tokenizer.decode(output, skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Q0: 너에 대해서 설명해봐.  A: 안녕하세요! 저는 한국에서 온 대학생이에요. 이름은 김민수이고, 20살이에요. 저는 컴퓨터 공학을 전공하고 있어요. 컴퓨터 프로그래밍과 인공지능에 관심이 많아요.  A: 반가워요! 김민수 군, 정말 멋지네요. 컴퓨터 공학을 전공하다니 대단해요. 인공지능에 대한 관심도 정말 대
Q1: 인공지능의 장점은? 인공지능(AI)은 다양한 분야에서 혁신적인 변화를 가져오고 있습니다. 그 장점은 다음과 같습니다:

1. **데이터 분석 및 예측**:
   - 대량의 데이터를 빠르고 정확하게 분석하여 패턴을 발견하고 예측할 수 있습니다.
   - 의료, 금융, 교통 등 다양한 분야에서 유용하게 활용됩니다.

2. **자동화 및 효율성**:
   - 반복적이고 시간
Q2: 동아대에 대해 알려줘
아주대와 함께 경상권을 대표하는 사립대학교 중 하나인 아시아대(아주대)는 다양한 학문 분야에서 우수한 교육을 제공하는 대학입니다. 다음은 아시아대(아주대)에 대한 주요 정보입니다:

### 학교 개요
- **학교명**: 아시아대학교(아주대)
- **소재지**: 경상남도 창원시
- **설립 연도**: 1997년

###


In [8]:
# 파인튜닝
tokens_seen, global_step = 0, -1

losses = []

optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001, weight_decay=0.01)

for epoch in range(10):
    model.train()  # 트레이닝 모드
    
    epoch_loss = 0
    for input_batch, target_batch in train_loader:
        optimizer.zero_grad() # Reset loss gradients from previous batch iteration
        input_batch, target_batch = input_batch.to(device), target_batch.to(device)

        logits = model(input_batch).logits # 뒤에 .logits를 붙여서 tensor만 가져옴

        loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
        epoch_loss += loss.item()
        loss.backward() # 손실 계산 후 역전파
        optimizer.step() # 가중치 업데이트트
        tokens_seen += input_batch.numel()
        global_step += 1

        print(f"{global_step} Tokens seen: {tokens_seen}")

        # if global_step % 1000 == 0:
        #     print(f"Tokens seen: {tokens_seen}")
        # Optional evaluation step

    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    print(f"Epoch: {epoch}, Loss: {avg_loss}")
    torch.save(model.state_dict(), "model_" + str(epoch).zfill(3) + ".pth")

0 Tokens seen: 118
1 Tokens seen: 236


KeyboardInterrupt: 

In [ ]:
# 파인튜닝 후에 어떻게 응답하는지 확인

model.load_state_dict(torch.load("model_009.pth", map_location=device, weights_only=True))
model.eval()

In [ ]:
# 학습 그래프
import matplotlib.pyplot as plt

plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.show()

In [ ]:
# 파인튜닝 후에 어떻게 대답하는지 확인
questions = [ qna['q_ids'] for qna in qna_list]

for i, q_ids in enumerate(questions):

    model.eval()
    with torch.no_grad():
        output = model.generate(
            torch.tensor([q_ids]).to("cuda"),
            max_new_tokens=32,
            #attention_mask = (input_ids != 0).long(),
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            # temperature=1.2,
            # top_k=5
        )

    output_list = output.tolist()
    print(f"Q{i}: {tokenizer.decode(output[0], skip_special_tokens=True)}")